## Import packages and functions

In [326]:
import requests
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import plotly.graph_objs as go
import plotly.plotly as py
from datetime import date
from data import get_booli_data

def get_sales_data():

    try:
        sales_data = pd.read_csv('booli_data ' + str(date.today())+'.csv', low_memory=True)
    except:
        sales_data = get_booli_data()
        sales_data.to_csv('booli_data ' + str(date.today()) + '.csv', index=False, encoding="utf-8-sig")
    return sales_data

def get_averages(df):

    df_averages = pd.DataFrame(columns=['months', 'averagePrices', 'deals'])
    df_averages['months'] = pd.date_range(start = df['soldDate'].min(), end=df['soldDate'].max(), freq='M')
    i = 0
    for month in df_averages['months']:
        #print(i)
        if i == 0:
            filtered_df = df[df['soldDate'] < str(month)]
            filtered_df['squareMeterPrice'].dropna(inplace=True)
            df_averages.at[i,'averagePrices'] = filtered_df['squareMeterPrice'].mean()
            df_averages.at[i,'deals'] = len(filtered_df)
            last_month = month
            i = i + 1
            continue
        filtered_df = df[(df['soldDate'] >= str(last_month)) & (df['soldDate'] < str(month))]
        filtered_df['squareMeterPrice'].dropna(inplace=True)
        df_averages.at[i,'averagePrices'] = filtered_df['squareMeterPrice'].mean()
        df_averages.at[i,'deals'] = len(filtered_df)
        last_month = month
        i = i + 1

    return df_averages

In [327]:
sales_data = get_sales_data()

Vasastan
Östermalm
Gärdet
Södermalm
Kungsholmen


## Filter received data and get averages

In [328]:
sales_data.drop(sales_data.index[sales_data['Area'] == 'Kungsholmen'], inplace=True)
sales_data.drop(sales_data.index[sales_data['Area'] == 'Gärdet'], inplace=True)
sales_data.drop(sales_data.index[sales_data['Area'] == 'Södermalm'], inplace=True)
sales_data.drop(sales_data.index[sales_data['Area'] == 'Other'], inplace=True)

sales_data.drop(sales_data.index[sales_data['soldDate']<'2018-06-01'], inplace=True)


one_room = sales_data[sales_data['rooms'] == 1]
two_room = sales_data[(sales_data['rooms'] >= 2) & (sales_data['rooms'] < 3)]
three_room = sales_data[(sales_data['rooms'] >= 3) & (sales_data['rooms'] < 4)]



one_room = one_room[(one_room['livingArea'] >= 20) & (one_room['livingArea'] < 30)]
two_room = two_room[two_room['soldPrice'] < 15000000]
two_room = two_room[(two_room['livingArea'] >= 60) & (two_room['livingArea'] < 80)]

three_room = three_room[three_room['soldPrice'] < 15000000]
three_room = three_room[(three_room['livingArea'] >= 65) & (three_room['livingArea'] < 80)]

average_one_room = get_averages(one_room)
average_two_room = get_averages(two_room)
average_three_room = get_averages(three_room)

# Dataframe descriptions

## One room

In [329]:
one_room[['listPrice', 'livingArea', 'rent', 'soldPrice','squareMeterPrice', 'priceIncrease']].describe(percentiles=[
    0.25,0.5,0.75,0.90,0.95,0.99])

,listPrice,livingArea,rent,soldPrice,squareMeterPrice,priceIncrease
count,2.040000e+02,204.000000,204.000000,2.040000e+02,204.000000,204.000000
mean,2.488578e+06,25.441667,1281.196078,2.844833e+06,112586.081180,1.147628
std,2.602342e+05,2.748467,396.581626,2.818689e+05,11990.286924,0.093300
min,1.995000e+06,20.000000,0.000000,2.250000e+06,86071.428571,0.928571
25%,2.295000e+06,23.375000,1004.750000,2.627500e+06,104000.000000,1.085595
50%,2.495000e+06,26.000000,1307.000000,2.800000e+06,111055.555556,1.142285
75%,2.650000e+06,28.000000,1529.250000,3.021250e+06,120000.000000,1.211755
90%,2.795000e+06,29.000000,1733.000000,3.234000e+06,128392.053973,1.275626
95%,2.938750e+06,29.000000,1847.000000,3.347000e+06,133775.000000,1.302020
99%,3.148500e+06,29.000000,2226.740000,3.694000e+06,144925.000000,1.332592


## Two room

In [317]:
two_room[['listPrice', 'livingArea', 'rent', 'soldPrice','squareMeterPrice', 'priceIncrease']].describe(percentiles=[
    0.25,0.5,0.75,0.90,0.95,0.99])

,listPrice,livingArea,rent,soldPrice,squareMeterPrice,priceIncrease
count,3.170000e+02,323.000000,323.000000,3.230000e+02,323.000000,317.000000
mean,5.569981e+06,65.615480,2706.925697,5.884087e+06,89795.173006,1.056848
std,8.721142e+05,4.710344,712.340762,9.377151e+05,13429.086086,0.080069
min,3.495000e+06,60.000000,0.000000,3.600000e+06,51333.333333,0.775862
25%,4.995000e+06,62.000000,2251.500000,5.200000e+06,80645.161290,1.000834
50%,5.495000e+06,64.000000,2741.000000,5.835000e+06,89354.838710,1.048159
75%,5.995000e+06,69.000000,3212.000000,6.450000e+06,97321.428571,1.101224
90%,6.668000e+06,72.800000,3576.800000,7.000000e+06,107553.143374,1.156520
95%,7.016000e+06,75.000000,3788.000000,7.390000e+06,113096.774194,1.185456
99%,8.202000e+06,77.000000,4007.400000,8.472500e+06,122914.285714,1.304170


## Three room

In [330]:
three_room[['listPrice', 'livingArea', 'rent', 'soldPrice','squareMeterPrice', 'priceIncrease']].describe(percentiles=[
    0.25,0.5,0.75,0.90,0.95,0.99])

,listPrice,livingArea,rent,soldPrice,squareMeterPrice,priceIncrease
count,2.480000e+02,250.000000,250.000000,2.500000e+02,250.000000,248.000000
mean,6.245605e+06,72.687600,3021.084000,6.610190e+06,90955.865311,1.059857
std,8.533487e+05,4.240099,783.347639,9.387130e+05,11817.094743,0.077576
min,4.295000e+06,65.000000,0.000000,4.200000e+06,58577.405858,0.782530
25%,5.698750e+06,70.000000,2557.250000,6.000000e+06,83110.301860,1.000000
50%,6.125000e+06,73.000000,3062.000000,6.502500e+06,91052.691090,1.053105
75%,6.750000e+06,77.000000,3492.500000,7.200000e+06,98007.982583,1.100906
90%,7.382500e+06,78.000000,3925.400000,7.802500e+06,105300.222387,1.159404
95%,7.661750e+06,79.000000,4351.150000,8.200000e+06,110141.025641,1.184338
99%,8.809350e+06,79.000000,4767.000000,9.051000e+06,122306.799337,1.243908


# Ta fram histogram för vå:or och tre:or

In [331]:
trace1 = go.Histogram(
    x=two_room['soldPrice'],
    #histnorm='percent',
    name='Två:or',
    xbins=dict(
        #start=-4.0,
        #end=3.0,
        #size=100000
    ),
    marker=dict(
        #color='#FFD7E9',
    ),
    opacity=0.75
)
trace2 = go.Histogram(
    x=three_room['soldPrice'],
    name='Tre:or',
    xbins=dict(
        #start=-4.0,
        #end=3.0,
        #size=100000
    ),
    marker=dict(
        #color='#EB89B5'
    ),
    opacity=0.75
)
data = [trace1, trace2]

layout = go.Layout(
    title='Histogram för två:or och tre:or i Vasastan och Östermalm - Slutpriser',
    xaxis=dict(
        title='Slutpris'
    ),
    yaxis=dict(
        title='Antal avslut'
    ),
    bargap=0.2,
    bargroupgap=0.1
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Histogram för två:or och tre:or i Vasastan och Östermalm - Slutpriser')

In [320]:
trace1 = go.Histogram(
    x=two_room['priceIncrease'],
    histnorm='percent',
    name='Två:or',
    xbins=dict(
        #start=-4.0,
        #end=3.0,
        #size=100000
    ),
    marker=dict(
        #color='#FFD7E9',
    ),
    opacity=0.75
)
trace2 = go.Histogram(
    x=three_room['priceIncrease'],
    histnorm='percent',
    name='Tre:or',
    xbins=dict(
        #start=-4.0,
        #end=3.0,
        #size=100000
    ),
    marker=dict(
        #color='#EB89B5'
    ),
    opacity=0.75
)
data = [trace1, trace2]

layout = go.Layout(
    title='Histogram för två:or och tre:or i Vasastan och Östermalm - Prisökning',
    xaxis=dict(
        title='Prisökning'
    ),
    yaxis=dict(
        title='Antal avslut'
    ),
    bargap=0.2,
    bargroupgap=0.1
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Histogram för två:or och tre:or i Vasastan och Östermalm - Slutpriser')

In [321]:
trace = [go.Histogram(x=three_room['soldPrice'],
                      histnorm='probability',
                     cumulative=dict(enabled=True)
                      )]
layout = go.Layout(
    title='Kumulativt histogram för tre:or',
    xaxis=dict(
        title='Slutpris'
    ),
    yaxis=dict(
        title='Antal avslut'
    )
)
fig = go.Figure(data=trace, layout=layout)
py.iplot(fig, filename='Kumulativt histogram för tre:or')

In [322]:
trace = [go.Histogram(x=two_room['soldPrice'],
                      histnorm='probability',
                     cumulative=dict(enabled=True)
                      )]
layout = go.Layout(
    title='Kumulativt histogram för två:or',
    xaxis=dict(
        title='Slutpris'
    ),
    yaxis=dict(
        title='Antal avslut'
    )
)
fig = go.Figure(data=trace, layout=layout)
py.iplot(fig, filename='Kumulativt histogram för två:or')

In [323]:
trace1 = go.Scatter(
    x =average_two_room['months'],
    y =average_two_room['averagePrices'],
    mode = 'lines+markers',
    name='Snittpriser för två:or'
)

trace2 = go.Scatter(
    x =average_three_room['months'],
    y =average_three_room['averagePrices'],
    mode = 'lines+markers',
    name='Snittpriser för tre:or'
)

trace3 = go.Scatter(
    x =average_one_room['months'],
    y =average_one_room['averagePrices'],
    mode = 'lines+markers',
    name='Snittpriser för ett:or'
)


data = [trace1, trace2,trace3]

# Plot and embed in ipython notebook!
py.iplot(data, filename='Snittpriser - utveckling')

In [325]:
trace1 = go.Scatter(
    x =average_two_room['months'],
    y =average_two_room['deals'],
    mode = 'lines+markers',
    stackgroup='one',
    name='Avslut för två:or'
)

trace2 = go.Scatter(
    x =average_three_room['months'],
    y =average_three_room['deals'],
    mode = 'lines+markers',
    stackgroup='one',
    name='Avslut för tre:or'
)

trace3 = go.Scatter(
    x =average_one_room['months'],
    y =average_one_room['deals'],
    mode = 'lines+markers',
    stackgroup='one',
    name='Avslut för ett:or'
)


data = [trace1, trace2,trace3]

py.iplot(data, filename='Antal avslut')